In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
fullDataPath = "/content/gdrive/My Drive/DS532_Text_Analytic/Homework/songData/";

In [4]:
import glob
import nltk
import pandas as pd
import numpy as np 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
files = glob.glob(fullDataPath + "*.txt")
corpus = []
songs = [file.split("/")[-1].split(".")[0] for file in files]

for file in files:
  f = open(file, "r", encoding = "utf-8")
  corpus.append(f.read())
  f.close()

# songtok = [nltk.tokenize.word_tokenize(song) for song in corpus]

In [6]:
print(songs)

['dream - priscilla ahn', 'love songs on the radio - mojave', 'sitting, waiting, wishing - johnson jack', 'you got growin up to do - joshua radin', 'blow away - a fine frenzy', 'dragon tales theme song - dragon tales', 'pull my heart away - jack penate', 'eet - regina spektor', 'the gardener - the tallest man on earth', 'blue skies - noah and the whale', 'low rising - the swell season', 'body like a back road - sam hunt', 'dirt on my boots - jon pardi', 'the fighter - keith urban', 'hurricane - luke combs', 'believer - imagine dragons', 'heavy - linkin park', "human - rag'n'bone man", 'let me out - gorillaz', 'sucker for pain - lil wayne, wiz khalifa & imagine dragons', 'even if - mercyme', 'when i pray for you - dan + shay', 'oh my soul - casting crowns', 'i have this hope - tenth avenue north', 'oceans (where feet may fail) - hillsong', 'break up to make up - jeremih', 'billie jean - michael jackson', 'beat it - michael jackson', 'halo - beyonce knowles', 'humble - kendrick lamar', "

Vector Representation with TFIDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist

In [0]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
tfidf = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names()).transpose()

In [0]:
def queryTFIDF(query):
  # query vector
  query_vector = vectorizer.transform([query])
  q = pd.DataFrame(query_vector.toarray(), columns = vectorizer.get_feature_names()).transpose()

  sim = 1 - cdist(tfidf.values.T, query_vector.toarray(), metric = "cosine")
  songSuggest = [songs[i] if sim[i] == np.max(sim) else None for i in range(len(sim))]
  songSuggest = list(filter(None, songSuggest))

  #print(q)

  return songSuggest;

Vector Represent with Inverted Text

In [10]:
print(corpus)

["priscilla ahn a good day dream i was a little girl alone in my little world who dreamed of a little home for me.  i played pretend between the trees, and fed my houseguests bark and leaves, and laughed in my pretty bed of green.   i had a dream  that i could fly from the highest swing.  i had a dream.   long walks in the dark through woods grown behind the park, i asked god who i'm supposed to be.  the stars smiled down on me, god answered in silent reverie. i said a prayer and fell asleep.   i had a dream  that i could fly from the highest tree.  i had a dream.   now i'm old and feeling grey. i don't know what's left to say about this life i'm willing to leave.  i lived it full and i lived it well, there's many tales i've lived to tell. i'm ready now, i'm ready now, i'm ready now to fly from the highest wing.   i had a dream", "mojave love songs on the radio love songs on the radio she looks just like an angel  when she walks across the room  she shines tonight  her golden light  is

In [0]:
sentok = [word_tokenize(sentence.lower()) for sentence in corpus]
words = list(sorted(set([word.lower() for wordlist in sentok for word in wordlist])))

In [0]:
# remove stop word
words = [word.lower() for word in words if word not in nltk.corpus.stopwords.words("english")]

In [0]:
inverted_index = []
for sen in sentok:
  d = [True if word in sen else False for word in words]
  inverted_index.append(d)

df = pd.DataFrame(np.array(inverted_index).T, columns = [songs[i] for i in range(len(corpus))])
df["words"] = words
df = df.set_index("words")

In [14]:
df.head(10)

,dream - priscilla ahn,love songs on the radio - mojave,"sitting, waiting, wishing - johnson jack",you got growin up to do - joshua radin,blow away - a fine frenzy,dragon tales theme song - dragon tales,pull my heart away - jack penate,eet - regina spektor,the gardener - the tallest man on earth,blue skies - noah and the whale,low rising - the swell season,body like a back road - sam hunt,dirt on my boots - jon pardi,the fighter - keith urban,hurricane - luke combs,believer - imagine dragons,heavy - linkin park,human - rag'n'bone man,let me out - gorillaz,"sucker for pain - lil wayne, wiz khalifa & imagine dragons",even if - mercyme,when i pray for you - dan + shay,oh my soul - casting crowns,i have this hope - tenth avenue north,oceans (where feet may fail) - hillsong,break up to make up - jeremih,billie jean - michael jackson,beat it - michael jackson,halo - beyonce knowles,humble - kendrick lamar,that's what i like - bruno mars,dna - kendrick lamar,mask off - future,ispy (feat lil yachty) - kyle,hey baby (feat deb's daughter) - dimitri vegas & like mike & diplo,chained to the rhythm (feat skip marley) - katy perry,find me (feat birdy) - sigma,alone - alan walker,trouble - offaiah,despacito (feat daddy yankee) - luis fonsi,...,je suis de retour - niagara,amdjer de nã´s terra - cesaria evora,aqui para voces - buraka som sistema,straight jeans & fitted - vybz kartel,bright side of life - rebelution,one love - playing for change,lazy afternoon - rebelution,michael jackson - negativland,ligea - team sleep,twin of myself - black moth super rainbow,mr clicketty cane - peter combe,fruit salad - the wiggles,miss independent - superstar kidz,toffee apple - peter combe,newspaper mama - peter combe,mr sun - barney,me and my teddy - barney,barney espanol - barney,ren's pecs - ren and stimpy,the weird al show theme - weird al yankovic,the thanksgiving song performed by adam sandler - adam sandler,show me your genitals 2:e=mc vagina - jon lajoie,å vi e aik - aik trubaduren,my body's a zombie for you - dead man's bones,milord - piaf édith,"ki-i-youdleing dog, the (4 apr 1913) - berlin irving",in the room where you sleep - dead man's bones,c'est si bon (it's so good) - barbra streisand,el papa de los pollitos - los tucanes de tijuana,zombie jamboree - rockapella,come rain or come shine - bette midler,a todos los santos - palomo,dalam hati saja - warna,movement 5 - vangelis,nei tuoi occhi - yanni,the show goes on - hornsby bruce,keep me hanging on - kim wilde,a letter from joey - easton sheena,no sound but a heart - easton sheena,sos - cetera peter
words,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
!,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,...,False,False,True,True,False,False,False,True,False,False,True,True,False,False,False,True,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False
&,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
',False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,True,True,False,True,False,False,False,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,

In [0]:
invidx = df.copy()

In [0]:
def queryInvertedIndex(query, invidx):
  words = invidx.index
  query_token = word_tokenize(query.lower())

  # remove stop word
  query_token = [word.lower() for word in query_token if word not in nltk.corpus.stopwords.words("english")]

  query_vector = [True if word in query_token else False for word in words]

  docs = [doc if any(np.array(query_vector) & invidx[doc].values) else None for doc in invidx.columns]
  docs = list(filter(None, docs))
  return docs

Tell me how you feel

In [20]:
howRU = input("your feel? ")

your feel? Love Chunlee


In [0]:
songTFIDF = queryTFIDF(howRU);
songInvertedIndex = queryInvertedIndex(howRU, invidx)

In [22]:
print("(TFIDF)Your song is ")
print(songTFIDF)
print("")
print("(InvertedIndex)Your song is ")
songInvertedIndex

(TFIDF)Your song is 
['crazy love - michael buble']

(InvertedIndex)Your song is 


['love songs on the radio - mojave',
 'sitting, waiting, wishing - johnson jack',
 'blue skies - noah and the whale',
 'low rising - the swell season',
 'the fighter - keith urban',
 'believer - imagine dragons',
 'sucker for pain - lil wayne, wiz khalifa & imagine dragons',
 'when i pray for you - dan + shay',
 'break up to make up - jeremih',
 'billie jean - michael jackson',
 'find me (feat birdy) - sigma',
 'let the four winds blow - roy brown',
 'slow marching band - jethro tull',
 'dog days are over - florence',
 'hustle - paris',
 'cry me a river - michael buble',
 'crazy love - michael buble',
 'hold on - michael buble',
 'whatever it takes - michael buble',
 '    shape of you - ed sheeran',
 'i feel it coming - the weeknd',
 'rockin robin - day bobby',
 'where is the love - hathaway donny',
 'stronger - hillsong',
 'bye bye blackbird - cantor eddie',
 'the hawaiian wedding song  ( ke kali nei au ) - perry como',
 'god help the outcasts - alan menken',
 'no letting go (remix)"(